In [1]:

%pprint
%run ../load_magic/storage.py

s = Storage()

Pretty printing has been turned OFF


In [2]:

def get_proportion_score(word_str, letter_proportions_df):
    word_score = 0
    for row_index, row_series in letter_proportions_df.iterrows():
        letter = row_series.letter_char
        if letter in word_str:
            proportion = row_series.proportion
            word_score += proportion
    
    return word_score

In [16]:

def build_lists(candidate_word, colors_list, greys_list=None, yellows_list=None, greens_list=None):
    if greys_list is None: greys_list = []
    if yellows_list is None: yellows_list = []
    if greens_list is None: greens_list = []
    for color, index in zip(range(len(candidate_word)), colors_list):
        if color == 'grey':
            greys_list.append(candidate_word[index])
        elif color == 'yellow':
            yellows_list.append(candidate_word[index])
        elif color == 'green':
            greens_list.append(f"(candidate_word[{index}] == '{candidate_word[index]}')")
    
    return greys_list, yellows_list, greens_list

In [3]:

def get_best_word(candidates_list):
    
    # Get proportions from candidate list
    import collections

    letters_list = []
    for word_str in candidates_list:
        letters_list += list(word_str)
    counter = collections.Counter(letters_list)
    letter_proportions_df = pd.DataFrame([{'letter_char': x, 'letter_count': y} for x, y in dict(counter).items()])
    min_count = letter_proportions_df.letter_count.min()
    letter_proportions_df['proportion'] = letter_proportions_df.letter_count.map(lambda x: x/min_count)
    
    # Minimize words to pick through after you get the next word colors back
    
    # Calculate maximum proportionality
    max_score = 0
    max_word = ''
    for word_str in candidates_list:
        word_score = get_proportion_score(word_str, letter_proportions_df)
        if word_score > max_score:
            max_score = word_score
            max_word = word_str
    
    return max_word, max_score

In [4]:

# Get words from the Wordle JavaScript
if s.pickle_exists('wordle_words_list'):
    words_list = s.load_object('wordle_words_list')
else:

    # Get words from the Wordle JavaScript
    import requests

    link = 'https://www.powerlanguage.co.uk/wordle/main.db1931a8.js'
    f = requests.get(link)
    commands_list = f.text.split(';')
    list_str = commands_list[331].split(']')[0].split('[')[1]
    words_list = eval(f'[{list_str}]')
    s.store_objects(wordle_words_list=words_list)

In [5]:

if s.pickle_exists('english_letter_frequencies_df'):
    letter_proportions_df = s.load_object('english_letter_frequencies_df')
else:
    
    # Get proportions from word list
    import collections

    letters_list = []
    for word_str in words_list:
        letters_list += list(word_str)
    counter = collections.Counter(letters_list)
    letter_proportions_df = pd.DataFrame([{'letter_char': x, 'letter_count': y} for x, y in dict(counter).items()])
    min_count = letter_proportions_df.letter_count.min()
    letter_proportions_df['proportion'] = letter_proportions_df.letter_count.map(lambda x: x/min_count)
    s.store_objects(english_letter_frequencies_df=letter_proportions_df)


----
# Solve a Wordle

In [19]:

# Get words from GloVe: Global Vectors for Word Representation
import re

words_list = []
file_path = r'D:\Documents\GitHub\job-hunting\data\6B\glove.6B.300d.txt'
az_regex = re.compile('[^a-z]')
with open(file_path, encoding='utf8') as infile:
    for line in infile:
        word_str = line.split()[0]
        if (len(word_str) == 5) and not az_regex.search(word_str):
            words_list.append(word_str)
words_list.remove('aerio')
words_list.remove('ioane')
words_list.remove('eonia')
words_list.remove('tabou')
words_list.remove('tamou')
words_list.remove('tarom')
words_list.remove('tarmo')
words_list.remove('marot')

In [49]:

words_list.remove('marto')

In [32]:

max_word, max_score = get_best_word(words_list)
max_word, max_score

('raise', 188.09339407744875)

In [50]:

greys_list = ['i', 's', 'e']
yellows_list = ['r', 'a']
candidates_list = []
for word_str in words_list:
    if (word_str[0] not in ['r']) and (word_str[1] == 'a') and (word_str[2] not in ['i']):
        if (word_str[3] not in ['s']) and (word_str[4] not in ['e']):
            if all(map(lambda x: x in word_str, yellows_list)):
                if all(map(lambda x: x not in word_str, greys_list)):
                    candidates_list.append(word_str)
max_word, max_score = get_best_word(candidates_list)
print(max_word, sorted(candidates_list))

tabor ['aabar', 'aamar', 'aarau', 'aaron', 'aaryn', 'baard', 'baarn', 'babar', 'babor', 'babra', 'babur', 'bacar', 'badar', 'bador', 'badra', 'badru', 'badry', 'bagar', 'bagra', 'bahar', 'bahro', 'bahru', 'bajar', 'bajor', 'bajra', 'bajur', 'bakar', 'bakra', 'bakry', 'bakur', 'balar', 'baldr', 'balor', 'bamar', 'baqar', 'baraa', 'barab', 'barac', 'barad', 'barah', 'barak', 'baral', 'baram', 'baran', 'baraq', 'barat', 'barau', 'baray', 'barba', 'barbo', 'barbu', 'barby', 'barca', 'barch', 'barco', 'barda', 'bardo', 'bardu', 'bardy', 'barff', 'barga', 'bargh', 'bargo', 'bargy', 'barha', 'barka', 'barko', 'barla', 'barma', 'barmy', 'barna', 'barno', 'barnt', 'barny', 'baron', 'barot', 'barra', 'barro', 'barry', 'barta', 'barth', 'bartl', 'barto', 'barty', 'bartz', 'barua', 'barud', 'barum', 'barun', 'baruq', 'barva', 'barwa', 'barza', 'bator', 'batra', 'batur', 'batyr', 'bauru', 'bawra', 'bayar', 'bayor', 'bazar', 'cabra', 'cakra', 'calor', 'camra', 'camry', 'capra', 'caral', 'caram', 'ca

In [41]:

greys_list = ['i', 's', 'e', 'b', 'o']
yellows_list = ['r', 'a']
candidates_list = []
for word_str in words_list:
    if (word_str[0] == 't') and (word_str[1] == 'a') and (word_str[2] not in ['i']):
        if (word_str[3] not in ['s']) and (word_str[4] not in ['e']):
            if all(map(lambda x: x in word_str, yellows_list)):
                if all(map(lambda x: x not in word_str, greys_list)):
                    candidates_list.append(word_str)
max_word, max_score = get_best_word(candidates_list)
print(max_word, sorted(candidates_list))

taboo ['taboo', 'tabot', 'tacom', 'tatoo', 'tatou', 'tavoy']


In [35]:

greys_list = ['r', 'i', 'l', 'n', 't', 'd']
yellows_list = ['a', 's', 'e']
candidates_list = []
for word_str in words_list:
    if (word_str[0] == 's') and (word_str[1] not in ['a', 'e', 't']) and (word_str[2] == 'e'):
        if (word_str[3] == 'a') and (word_str[4] not in ['e', 's', 'd']):
            if all(map(lambda x: x in word_str, yellows_list)):
                if all(map(lambda x: x not in word_str, greys_list)):
                    candidates_list.append(word_str)
max_word, max_score = get_best_word(candidates_list)
print(max_word, sorted(candidates_list))

speak ['sheaf', 'speak']


In [24]:

greys_list = ['l', 'e', 'h', 'n', 'p']
yellows_list = ['a', 't', 'r', 's']
candidates_list = []
for word_str in words_list:
    if (word_str[0] == 't') and (word_str[1] == 'r') and (word_str[2] == 'a'):
        if (word_str[3] not in ['e', 's', 'n', 'p']) and (word_str[4] == 's'):
            if all(map(lambda x: x in word_str, yellows_list)):
                if all(map(lambda x: x not in word_str, greys_list)):
                    candidates_list.append(word_str)
max_word, max_score = get_best_word(candidates_list)
print(max_word, sorted(candidates_list))

trams ['tracs', 'traks', 'trams', 'trays']


In [25]:

greys_list = ['l', 'e', 'h', 'n', 'p', 'm']
yellows_list = ['a', 't', 'r', 's']
candidates_list = []
for word_str in words_list:
    if (word_str[0] == 't') and (word_str[1] == 'r') and (word_str[2] == 'a'):
        if (word_str[3] not in ['e', 's', 'n', 'p', 'm']) and (word_str[4] == 's'):
            if all(map(lambda x: x in word_str, yellows_list)):
                if all(map(lambda x: x not in word_str, greys_list)):
                    candidates_list.append(word_str)
max_word, max_score = get_best_word(candidates_list)
print(max_word, sorted(candidates_list))

trays ['tracs', 'traks', 'trays']


In [ ]:

def get_largest_equivalence_class(candidates_list, base_greys_list=None, base_yellows_list=None, base_greens_list=None):
    if base_greys_list is None: base_greys_list = []
    if base_yellows_list is None: base_yellows_list = []
    if base_greens_list is None: base_greens_list = []
    max_possibles_list = []
    max_candidate_word = ''
    for candidate_word in candidates_list:
        possibles_list = []
        for color1 in ['grey', 'yellow', 'green']:
            for color2 in ['grey', 'yellow', 'green']:
                for color3 in ['grey', 'yellow', 'green']:
                    for color4 in ['grey', 'yellow', 'green']:
                        for color5 in ['grey', 'yellow', 'green']:
                            colors_list = [color1, color2, color3, color4, color5]
                            greys_list, yellows_list, greens_list = build_lists(candidate_word, colors_list,
                                                                                base_greys_list, base_yellows_list, base_greens_list)
                            if all(map(lambda x: eval(x), greens_list)):
                                if all(map(lambda x: x in word_str, yellows_list)):
                                    if all(map(lambda x: x not in word_str, greys_list)):
                                        possibles_list.append(candidate_word)

In [ ]:

for word_str in candidates_list:
    greys_list = ['a', 'i', 's']
    yellows_list = ['r']
    greens_list = ["(candidate_word[4] == 'e')"]
    max_possibles_list = get_largest_equivalence_class(word_str, base_greys_list=greys_list, base_yellows_list=yellows_list, base_greens_list=greens_list)

In [8]:

greys_list = ['l', 'a', 't', 'u']
yellows_list = ['r', 'g']
candidates_list = []
for word_str in words_list:
    if (word_str[0] not in ['l', 'r']) and (word_str[1] == 'o') and (word_str[2] not in ['t', 'g']):
        if (word_str[3] not in ['e', 'u']) and (word_str[4] == 'e'):
            if all(map(lambda x: x in word_str, yellows_list)):
                if all(map(lambda x: x not in word_str, greys_list)):
                    candidates_list.append(word_str)
max_word, max_score = get_best_word(candidates_list)
print(max_word, sorted(candidates_list))

forge ['forge', 'gorge']


In [9]:

greys_list = ['l', 'a', 't', 'u', 'f']
yellows_list = []
candidates_list = []
for word_str in words_list:
    if (word_str[0] not in ['l', 'r', 'f']) and (word_str[1] == 'o') and (word_str[2] == 'r'):
        if (word_str[3] == 'g') and (word_str[4] == 'e'):
            if all(map(lambda x: x in word_str, yellows_list)):
                if all(map(lambda x: x not in word_str, greys_list)):
                    candidates_list.append(word_str)
max_word, max_score = get_best_word(candidates_list)
print(max_word, sorted(candidates_list))

gorge ['gorge']



----
# Explore the Data

In [ ]:

import random

random.sample(words_list, 5)

In [ ]:

letter_proportions_df.sample(5)

In [ ]:

mask_series = (letter_proportions_df.letter_char == 'e')
letter_proportions_df[mask_series]

In [ ]:

letter_proportions_df.sort_values('proportion')


----

In [ ]:

if s.pickle_exists('english_word_frequencies_df'):
    word_proportions_df = s.load_object('english_word_frequencies_df')
else:
    
    # Get words from the Wordle JavaScript
    if s.pickle_exists('wordle_words_list'):
        words_list = s.load_object('wordle_words_list')
    else:

        # Get words from the Wordle JavaScript
        import requests

        link = 'https://www.powerlanguage.co.uk/wordle/main.db1931a8.js'
        f = requests.get(link)
        commands_list = f.text.split(';')
        list_str = commands_list[331].split(']')[0].split('[')[1]
        words_list = eval(f'[{list_str}]')
        s.store_objects(wordle_words_list=words_list)
        
    rows_list = [{'word_str': word_str, 'proportion_score': get_proportion_score(word_str)} for word_str in words_list]
    word_proportions_df = pd.DataFrame(rows_list)
    s.store_objects(english_word_frequencies_df=word_proportions_df)

In [ ]:

word_proportions_df.sample(5)

In [ ]:

proportion_score_dict = word_proportions_df.set_index('word_str').proportion_score.to_dict()